In [1]:
%load_ext autoreload
%load_ext autotime

In [2]:
%autoreload 2

In [3]:
import sys
sys.path.append('../../../')

# Controles Sintéticos utilizando GAMs
Modelos Aditivos Generalizados para el diseño de contrafactuales robustos.

## Un poco sobre mí

- Andrés Bucchi, VP de Data & Analytics @ Sodimac; previamente Uber HQ.
- Sodimac tiene presencia en 7 países; Data & Analytics cerca de 100 personas
- Tenemos múltiples proyectos interesantes, desde plataformas de Data hasta Analítica Avanzada, pero lo que más nos (me) gusta es la experimentación.
- Soy de profesión Ingeniero Comercial (BBA), pero mi pasión siempre fue el programar.

## Un poco sobre los Controles Sintéticos

Los controles sintéticos nos permiten realizar análisis causal cuando no existen otras alternativas (más robustas) para construir un contrafactual válido — e.g., muestreo aleatorio, muestreo temporal, cuasi-experimentos, etc.

Este tipo de análisis es de altísima utilidad al momento de evaluar el efecto de distintas políticas de intervención que no pueden "prenderse y apagarse" o no pueden ofrecérseles solamente a un grupo de individuos. Por ejemplo -

- Un sistema de atención de urgencia que podría salvar más vidas, lo que presenta un problema ético y reputacional.
- Mejores condiciones en créditos hipotecarios según un nuevo modelo de riesgo, si es que la ley no permite tener ofertas diferentes para un mismo producto, bajo mismas condiciones.
- Apertura de una nueva tienda y posible efecto en canales online.
- Entre __muchos__ otros...

## Un poco sobre GAMs

Generalized Additive Models (GAMs) son el resultado de una combinación lineal de Generalizaed Linear Models (GMLs). Permiten relacionar la __media__ de una serie de respuesta $Y$ a un vector de controles $X$, a través de una función de vínculo $g(·)$.

$$
g(E[y]) = f_0(x_0) + f_1(x_1) + ... + f_k(x_k)
$$

La función $g(·)$ puede tomar múltiples formas, y puede ser interpretada como la función inversa a la media de alguna familia de distribuciones — e.g., en el caso de una poisson, si la media $\mu$ puede ser expresada como una combinación lineal de $X$

$$
\mu=e^{X\beta}
$$

Entonces $g(·)$ es la función exponencial y $g^{-1}(·)$ es $ln(\mu)$.

### ¿Y qué?

El principal take-away es que los GAMs son muy flexibles y, en general, permiten construir forecasts con errores iid. que se comportan de una forma normalizada, agregando distintos niveles de tendencias y quiebres de tendencias, etc. Esto las convierte en muy buenas candidatas para el análisis contrafactual.

## Construyendo un contrafactual

Buscamos construir un contrafactual $\hat{y}_c$ con la finalidad de estimar el efecto $\delta{y}$ de un tratamiento, dado que

$$
\hat{ \delta{y} } = y - \hat{y}_c
$$

Nótese que $y$ es un valor conocido y observado tanto antes como después del tratamiento.

Utilizando GAMs, podemos reescribir lo anterior -

$$
\hat{ \delta{y} } = y - g^{-1}(f_0(x_0) + f_1(x_1) + ... + f_k(x_k))
$$

Donde $X$ representa un vector de controles, utilizado para construir nuestro contrafactual $\hat{y}_c$.

### Eligiendo los controles correctos

Al construir $\hat{y}_c$, lo que queremos evitar es la interferencia, ya que no debiesen existir cualquier tipo de contaminación desde los controles hacia la respuesta y viceversa. Visualmente,

[<img src="assets/gamsc/good.png" width="600" style="display: block; margin-left: auto; margin-right: auto;"/>](assets/gamsc/good.png "The good")

Y lo que debemos evitar -

[<img src="assets/gamsc/bad.png" width="600" style="display: block; margin-left: auto; margin-right: auto;"/>](assets/gamsc/bad.png "The bad")

[<img src="assets/gamsc/also-bad.png" width="600" style="display: block; margin-left: auto; margin-right: auto;"/>](assets/gamsc/also-bad.png "The worse")

[<img src="assets/gamsc/i-quit.png" width="600" style="display: block; margin-left: auto; margin-right: auto;"/>](assets/gamsc/i-quit.png "The worst")

### Por ejemplo

Utilizar los volúmenes de abastecimiento para construir un contrafactual de ventas es...

- __Incorrecto__ — un mayor volumen de ventas incrementará la cantidad de pedidos de abastecimiento, por lo que la respuesta y el control tienen una relación causal. Utilizar el abastecimiento interno como control va a generar un feedback positivo -

        mayores ventas observadas
            >> mayores órdenes de abastecimiento
                >> predicción de un contrafactual más alto
                    >> subestimación del efecto del tratamiento

Utiliza la venta de otras tiendas para construir un contrafactual de ventas en una tienda distinta es...

- __Correcto__ — podemos asumir razonablemente que las ventas están principalmente afectadas por factores exógenos o causas comunes, independientes a un tratamiento, por lo que representan una buena alternativa para construir un contrafactual robusto.

## Otros beneficios de los GAMs

Los GAMs están ampliamente cubiertas por distintos paquetes en Python, varios de ellos bastante sofisticados y con una gran comunidad de soporte opensource. Mejor todavía - Facebook tiene un motor de forecasting llamado `Prophet` que permite, entre otras cosas, realizar muestreo posterior, ¡lo que facilita muchísimo la construcción de intervalos de confianza!

```bash
$ pip install prophet
```

[Documentación](https://facebook.github.io/prophet/).

## ¡Manos a la obra!

Empecemos por importar `fbprophet`. También utilizaremos algunas funcionalidades de `SodiMX`, el paquete de Experimentación de Sodimac — no es opensource todavía.

In [ ]:
import numpy as np
import pandas as pd
try:
    import fbprophet
except ImportError:
    import prophet as fbprophet
try:
    from sodimx.synthetic_control.utils import SyntheticData
except ImportError:
    from utils import SyntheticData
from matplotlib import pyplot as plt

plt.style.use('ggplot')

%matplotlib inline

Necesitamos un poco de data sintética, para la cuál, por diseño, además sabremos el efecto real.

In [ ]:
sd = SyntheticData(
    impulse=0.03,
    noise=5.0,
    bias=0.1,
    n_controls=20,
    n_informative_controls=10,
    n_samples=150,
    n_treatment=30
)
X, Y, slices = sd.make_arma()

$X$ es una matriz de series de tiempos de distintos controles, pero de los cuales solamente algunos son informativos, mientras otros tienen como objetivo el agregar ruido y probar la robustez de nuestro método.

In [ ]:
X.head()

$Y$ contiene la serie de tiempo de respuesta. Prophet requiere que existan las columnas `ds` e `y`, con las fechas de la serie y el valor de la respuesta, respectivamenente.

In [ ]:
Y.rename(columns={0: 'y'}, inplace=True)
Y['ds'] = Y.index
Y.head()

El objeto `slices` contiene los períodos de tratamiento y blackout. En este caso no vamos a utilizar períodos de blackout — podríamos querer eliminar días de lluvia, cybers, etc. — pero Prophet permite imputar automáticamente data faltante, por lo que es perfectamente válido remover (algunos) outliers.

In [ ]:
treatment_ixs = Y.loc[slices['treatment']].index
control_ixs = Y.index.difference(treatment_ixs)
slices

Visualmente, las series no muestran un claro quiebre en la respuesta $Y$ durante el período de control y tratamiento.

In [ ]:
fig, ax = plt.subplots(figsize=(12, 6))
ax.plot(Y.loc[control_ixs, 'y'], color='blue', label='control')
ax.plot(Y.loc[treatment_ixs, 'y'], color='red', label='treatment')
ax.vlines(x=control_ixs[-1],ymin=ax.dataLim.bounds[1],ymax=ax.dataLim.bounds[3],linestyle='--',alpha=0.5)
ax.legend();

### Utilizando Prophet

Veamos si logramos encontrar algún tipo de tendencia a través de `fbprophet`. Por ahora, solamente vamos a estar utilizando la serie de tiempo $Y$, apoyándonos en las capacidades de análisis de series de tiempo que `fbprophet` provee.

Para datasets pequelos, el proceso es relativamente rápido. Hasta ahora, solamente estaremos aprendiendo los parámetros más probables para nuestras GAMs. Más adelante utilizaremos muestreo sobre los posteriores, tanto para hacer forecasting como para construir intervalos de confianza.

In [ ]:
mdl = fbprophet.Prophet()
mdl.fit(Y.loc[control_ixs, :]);

Para construir forecasts necesitamos utilizar los métodos `make_future_dataframe` y `predict`.

In [ ]:
mdl.make_future_dataframe(30)

In [ ]:
_ = mdl.make_future_dataframe(30)
forecast = mdl.predict(_)
forecast.index = pd.DatetimeIndex(forecast['ds'], name='')
forecast

Los forecasts no son muy buenos. Definitivamente algo falta.

In [ ]:
fig, ax = plt.subplots(figsize=(12, 6))
ax.plot(Y.loc[control_ixs, 'y'], color='blue', label='control', alpha=0.3)
ax.plot(Y.loc[treatment_ixs, 'y'], color='red', label='treatment', alpha=0.3)
ax.vlines(x=control_ixs[-1], ymin=ax.dataLim.bounds[1], ymax=ax.dataLim.bounds[3], linestyle='--', alpha=0.5)
ax.plot(forecast.yhat, color='green', label='forecast')
ax.legend();

### Agreguemos controles

¡Los controles son informativos! Si los agregamos, vamos a mejorar nuestra capacidad de capturar efectos exógenos, que solamente agregan variabilidad no capturada a nuestra serie $Y$.

Entrenemos un segundo modelo, esta vez incluyendo todos los controles como regresores adicionales. Necesitaremos utilizar el método `add_regressor`, además de entregar los valores para cada uno de estos controles al momento de generar forecasts con `predict`.

In [ ]:
X.rename(columns={
    c: 'x{}'.format(c) for c in X.columns
}, inplace=True)
X.head()

In [ ]:
mdl2 = fbprophet.Prophet()
[
    mdl2.add_regressor(name=c) for c in X.columns
];

Para hacer las cosas más sencillas, creemos un único dataframe que contenga tanto la serie de respuesta como los controles, además de la columna `ds`, requerida por Prophet.

In [ ]:
XY = pd.concat([X, Y], axis=1, ignore_index=False)
XY.head()

Entrenemos el nuevo modelo.

In [ ]:
mdl2.fit(XY.loc[control_ixs, :]);

Para hacer un forecast, ahora tenemos pasar tanto los valores de los controles como la columna `ds`, en el mismo dataframe.

In [ ]:
Xt = mdl2.make_future_dataframe(30)
Xt.index = pd.DatetimeIndex(Xt.ds, name='')
Xt = pd.concat([Xt, X], axis=1)
Xt.tail()

In [ ]:
forecast2 = mdl2.predict(Xt)
forecast2.index = pd.DatetimeIndex(forecast2.ds, name='')
forecast2

¡Los resultados son mucho mejores!

In [ ]:
fig, ax = plt.subplots(figsize=(12, 6))
ax.plot(Y.loc[control_ixs, 'y'], color='blue', label='control', alpha=0.3)
ax.plot(Y.loc[treatment_ixs, 'y'], color='red', label='treatment', alpha=0.3)
ax.vlines(x=control_ixs[-1], ymin=ax.dataLim.bounds[1], ymax=ax.dataLim.bounds[3], linestyle='--', alpha=0.5)
ax.plot(forecast2.yhat, color='green', label='forecast2', alpha=0.3, marker='o')
ax.legend();

## Estimemos el efecto

Lo que nos interesa hacer es estimar el efecto e impulso total y lo las diferencias diarias, aunque podríamos medir diferencias significativas a nivel diario también. Para esto, necestiamos asegurar que estamos haciendo muestro posterior sobre series __acumuladas__ y no valores diarios.

Formalmente, el efecto total sería -

$$
\sum_{i\in{\ T}}{\delta{y}_i} = \sum_{i\in{\ T}}{(y_i - \hat{y}_i)}
$$

con $T$ representando el período de tratamiento.

### Acceso a muestreo posterior

Prophet entrega un fácil acceso a sampleo posterior del modelo, una vez entrenado. Podemos utilizar estas muestras no solamente para estimar el efecto punto a nivel diario, sino que también simular los intervalos de confianza. ¡Excelente!

In [ ]:
Yhat = mdl2.predictive_samples(Xt.loc[treatment_ixs, :])['yhat']
print(Y.loc[treatment_ixs, 'y'].shape, Yhat.shape)
Yhat

Podemos utilizar estas muestras para construir series de diferencias diarias entre observado $y_t$ y contrafactual $\hat{y}_t$, lo que nos permitirá calcular diferentes estadísticas sobre las series cumulativas.

In [ ]:
# observed response
yt = Y.loc[treatment_ixs, ['y']].values
print(yt.shape)
yt[:10]

El contrafactual está marginalmente bajo el observado. Ya sabemos que esto es correcto, dado que conocemos el impulso real.

In [ ]:
fig, ax = plt.subplots(figsize=(12, 6))
ax.set_title('Treatment period response')
pd.Series(Yhat.mean(axis=1), index=treatment_ixs).plot(label='counterfactual')
pd.Series(yt.flatten(), index=treatment_ixs).plot(ax=ax, label='observed')
ax.legend();

### Pero, ¿podremos detectar esto de forma estadísticamente significativa?

### ¡Sí!

In [ ]:
cdY = np.subtract(yt, Yhat).cumsum(axis=0)
# impulse
effect = pd.DataFrame(
    cdY.mean(axis=1)/Yhat.mean(axis=1).cumsum(),
    index=treatment_ixs,
    columns=['impulse_point']
)
l, u = np.percentile(cdY, [2.5, 97.5], axis=1)
effect['impulse_lb'] = l/Yhat.mean(axis=1).cumsum()
effect['impulse_ub'] = u/Yhat.mean(axis=1).cumsum()
effect = effect[['impulse_lb', 'impulse_point', 'impulse_ub']]

# total effect
effect['total_point'] = cdY.mean(axis=1)
effect['total_lb'] = l
effect['total_ub'] = u

print('Ground truth: {}'.format(sd.impulse))
pd.DataFrame([effect.iloc[-1][:3].to_dict()])

También podemos construir unos buenos gráficos para presentar los resultados de nuestra política de intervención, asegurando el "efecto wow".

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(15, 5))
axs[0].set_title('Cummulative effect')
axs[0].plot(effect.total_point)
axs[0].fill_between(x=effect.index, y1=effect.total_lb, y2=effect.total_ub, alpha=0.25)
axs[0].tick_params(rotation=45)
axs[1].set_title('Cummulative impulse')
axs[1].plot(effect.impulse_point)
axs[1].fill_between(x=effect.index, y1=effect.impulse_lb, y2=effect.impulse_ub, alpha=0.25)
axs[1].tick_params(rotation=45)

fig.tight_layout()

# Algunas ideas para cerrar

- Los Controles Sintéticos nos permiten realizar análisis causal sobre procesos en los que no podemos implementar experimentos válidos, incluso no aleatorios.
- Utilizar controles es una gran forma de reducir el ruido en nuestras series, aislando mejor los efectos causales de un tratamiento, y mejorando nuestras estimaciones de los efectos.
- Los GAMs son un gran candidato para construir contrafactuales robustos, ya que son muy flexibles y tienen bastante soporte en Python — e.g., Facebook Prophet.
- El proceso, por lo general, es muy bajo en consumo computacional, por lo que es recomendable realizar análisis de sensibilidad sobre los resultados — agregar o quitar días, mover el período de intervención un día hacia adelante y atrás, agregar y sacar controles, etc.

# ¡Gracias!
- ¿Preguntas?
- Pueden descargar el código [aquí](https://github.com/misterte/talks).